# Computer Vision

## Assignment - UNet

### Install

In [11]:
%pip install segmentation_models --quiet
%pip install comet_ml --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Comet Config

In [12]:
from comet_ml import Experiment

experiment = Experiment(project_name = "cv-deep")

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/mamello-justice/cv-deep/c7d2dd81940c429ea08f6e7b25b835ce
COMET INFO:   Parameters:
COMET INFO:     augmented  : True
COMET INFO:     batch_size : 2
COMET INFO:     epochs     : 2
COMET INFO:     features   : ['rgb']
COMET INFO:     input_size : [768, 1024]
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (1.53 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finis

### Imports

In [13]:
import time
import os
from os import path

import numpy as np

import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from segmentation_models import Unet

from cv_gmm_deep.datasets import load_data
from cv_gmm_deep.common import split_data, preprocess_data

print(f"numpy=={np.__version__}")
print(f"tensorflow=={tf.__version__}")
print(f"keras=={keras.__version__}")

numpy==1.23.4
tensorflow==2.10.0
keras==2.10.0


### Constants

In [14]:
BACKBONE = 'vgg16'

assets_dir = './assets'
data_dir = './data'

default_input_size = [768, 1024]
default_input_dir = path.join(
    assets_dir,
    f'puzzle_corners_{default_input_size[1]}x{default_input_size[0]}')

time_now = int(time.time())

default_cp_path = path.join(data_dir, str(time_now), 'cp.ckpt')
default_model_path = path.join(data_dir, str(time_now), 'model.h5')

### Params

In [15]:
params = {
    'augmented': True,
    'batch_size': 2,
    'epochs': 2,
    'features': ['rgb'],
    'input_size': default_input_size,
}

experiment.log_parameters(params)

### Setup

In [16]:
def setup_args():
    return {
        'cp_path': default_cp_path,
        'input_dir': default_input_dir,
        'model_path': default_model_path,
        'update_model': None,
        'cpu': True
    }

### Train

In [17]:
args = setup_args()

use_cpu = args['cpu']
if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

augmented = params['augmented']
batch_size = params['batch_size']
epochs = params['epochs']
features = params['features']
input_size = params['input_size']

cp_path = args['cp_path']
input_dir = args['input_dir']
model_path = args['model_path']
update_model = args['update_model']

height, width = input_size

In [18]:
raw_x, raw_y = load_data(input_dir, height, width)

Reading in masks: 100%|██████████| 48/48 [00:00<00:00, 296.17it/s]


In [19]:
data_x, data_y = preprocess_data(raw_x, raw_y, features)

In [20]:
(train_x, train_y), (val_x, val_y), (test_x, test_y) =\
    split_data(data_x, data_y, ratios=[0.7, .15, .15])
    
experiment.log_dataset_hash(train_x)

In [21]:
checkpoint_cb = ModelCheckpoint(filepath=cp_path,
                                save_weights_only=True,
                                verbose=1)

if update_model and model_path:
    model = load_model(model_path)
else:
    model = Unet(BACKBONE,
                 encoder_weights='imagenet',
                 input_shape=(*input_size, 3))

assert model is not None, "Could not load model"

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(val_x, val_y),
          callbacks=[checkpoint_cb])

if model_path is not None:
    model.save(model_path)


COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/2
17/17 [==============================] - ETA: 0s - loss: 0.2953 - accuracy: 0.9116 
Epoch 1: saving model to ./data\1667442801\cp.ckpt
17/17 [==============================] - 284s 17s/step - loss: 0.2953 - accuracy: 0.9116 - val_loss: 111.0625 - val_accuracy: 0.2370
Epoch 2/2
17/17 [==============================] - ETA: 0s - loss: 0.1188 - accuracy: 0.9739 
Epoch 2: saving model to ./data\1667442801\cp.ckpt
17/17 [==============================] - 279s 16s/step - loss: 0.1188 - accuracy: 0.9739 - val_loss: 19.8885 - val_accuracy: 0.2390


### Comet Commit

In [22]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/mamello-justice/cv-deep/ef33f23a837341588ba64405069fb9c3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [2]                : (0.911558210849762, 0.9738996028900146)
COMET INFO:     batch_accuracy [4]          : (0.5781771540641785, 0.975982666015625)
COMET INFO:     batch_loss [4]              : (0.1325690597295761, 0.7361920475959778)
COMET INFO:     epoch_duration [2]          : (279.46899999998277, 284.344000000041)
COMET INFO:     loss [2]                    : (0.11882644146680832, 0.29529324173927307)
COMET INFO:     val_accuracy [2]            : (0.23695901036262512, 0.2390144020318985)
COMET INFO:     val_loss [2]                : (19.888463973999023, 111.06253814697266)
COMET INFO:     validate_batch_accuracy [2] : (0.